In [10]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ta
from ta.trend import EMAIndicator
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import joblib
from tensorflow import keras

In [11]:
class DataLoader:
    def __init__(self,path):
        self.path=path
        
    @staticmethod
    def succesfull():
        print("✅Data Loaded succesfully")
        
    def load_data(self):
        data=pd.read_csv(self.path,sep="\t",names=["date","open","high","low","close","volume"])
        data["date"]=pd.to_datetime(data["date"],format="%Y-%m-%d %H:%M")
        data.set_index("date",inplace=True)
        self.succesfull()
        print("👾Shape of Data : ",data.shape)
        return data

class Features:
   
    @staticmethod
    def succesfull():
        print("✅Features Created succesfully")
        
    def make_features(self,data):
        data["ema"]=EMAIndicator(data["close"],window=21).ema_indicator()
        data["rsi"]=RSIIndicator(data["close"],window=13).rsi()
        bb=BollingerBands(data["close"])
        data["uperband"]=bb.bollinger_hband()
        data["lowerband"]=bb.bollinger_lband()
        data["bb_avg"]=bb.bollinger_mavg()
        data["mean_price"]=data["close"].rolling(13).mean()
        candle_mean=(data["high"]-data["low"]).mean()
        data["candle_strentgh"]=(data["high"]-data["low"])/candle_mean
        
        data.dropna(inplace=True)
        data.drop(["open","high","low"],axis=1,inplace=True)
        print("🏹 Shape Now is : ",data.shape)
        return data

class Scaling_Timeteps:
    
    @staticmethod
    def succesfull1():
        print("✅Data Scaled succesfully")
    @staticmethod
    def succesfull2():
        print("✅Timestep Added succesfully")
        
    def start_scaling(self,data,timesteps,future):
        print("-"*50)
        print("🔎 All features : ",data.columns)
        print("-"*50)
        features=data.values
        column=data.columns
        rest_columns=[]
        for idx in range(len(column)):
            if column[idx]=="close":
                forecast=idx
            else:
                rest_columns.append(idx)
        scaler1=MinMaxScaler(feature_range=(0,1))
        scaler2=MinMaxScaler(feature_range=(0,1))

        split=int(features.shape[0]*0.8)
        
        
        features[:split,forecast]=scaler1.fit_transform(features[:split,forecast].reshape(-1,1)).ravel()
        features[split:,forecast]=scaler1.transform(features[split:,forecast].reshape(-1,1)).ravel()
        features[:split,rest_columns]=scaler2.fit_transform(features[:split,rest_columns])
        features[split:,rest_columns]=scaler2.transform(features[split:,rest_columns])
        
        
        self.succesfull1()
        joblib.dump(scaler1,"gru_target.pkl")
        joblib.dump(scaler2,"gru_features.pkl")
        print("🤹🏻Succesfully stored MinMaxScaler")
        X_new=[]
        y_new=[]
        for i in range(len(features)-timesteps-future+1):
            X_new.append(features[i:i+timesteps])
            y_new.append(features[i+timesteps+future-1,forecast])
            
        self.succesfull2()

        X_new=np.array(X_new)
        y_new=np.array(y_new)
        print("❄️Shape of Train Data: ",X_new.shape)
        print("❄️Shape of Train Target : ",y_new.shape)
        
        return X_new,y_new

class Model_Training:
    
    def start_training(self,X,y):
        split=int(X.shape[0]*0.8)
        X_train=X[:split]
        X_test=X[split:]
        y_train=y[:split]
        y_test=y[split:]

        shape=X.shape[1:]
        model=keras.Sequential()
        model.add(keras.layers.GRU(128,activation="tanh",return_sequences=True,input_shape=shape))
        model.add(keras.layers.Dropout(0.2))
        model.add(keras.layers.GRU(128,activation="tanh",return_sequences=True))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.GRU(64,activation="tanh",return_sequences=True))
        model.add(keras.layers.Dropout(0.2))
        model.add(keras.layers.GRU(32,activation="tanh"))

        model.add(keras.layers.Dense(64,activation="relu"))
        model.add(keras.layers.Dropout(0.2))
        model.add(keras.layers.Dense(1,activation="linear"))

        model.compile(loss="mse",optimizer="adam",metrics=["mae"])
        callbacks=callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                          min_delta=0.0001,
                                          patience=10,
                                          verbose=1,
                                          mode="min",restore_best_weights=True)
        print(model.summary())
        model.fit(X_train,y_train,epochs=50,
                           validation_data=(X_test,y_test),callbacks=callbacks)

        model.save("gru_model.h5")
        print("✅Model Succesfully Trained and Store as 'lstm_model.h5' ")

In [12]:
def main():
    # Loading data from path
    loader = DataLoader("GBPUSD240.csv")
    df = loader.load_data()
    
    #Feature Engineering
    features=Features()
    df=features.make_features(df)

    #model
    timesteps=100
    print("_"*50)
    future=1
    scaling_timestep=Scaling_Timeteps()
    X,y=scaling_timestep.start_scaling(df,timesteps,future)

    model_training=Model_Training()
    model_training.start_training(X,y)

In [13]:
if __name__ == "__main__":
    main()

✅Data Loaded succesfully
👾Shape of Data :  (25848, 5)
🏹 Shape Now is :  (25828, 9)
__________________________________________________
--------------------------------------------------
🔎 All features :  Index(['close', 'volume', 'ema', 'rsi', 'uperband', 'lowerband', 'bb_avg',
       'mean_price', 'candle_strentgh'],
      dtype='object')
--------------------------------------------------
✅Data Scaled succesfully
🤹🏻Succesfully stored MinMaxScaler
✅Timestep Added succesfully
❄️Shape of Train Data:  (25728, 100, 9)
❄️Shape of Train Target :  (25728,)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                          │ (None, 100, 128)            │          53,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 100, 128)            │          99,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 100, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_6 (GRU)                          │ (None, 100, 64)             │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 100, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_7 (GRU)                          │ (None, 32)                  │           9,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 201,793 (788.25 KB)

 Trainable params: 201,537 (787.25 KB)

 Non-trainable params: 256 (1.00 KB)

None
Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 243s 347ms/step - loss: 0.0267 - mae: 0.1114 - val_loss: 3.6096e-04 - val_mae: 0.0150
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 209s 324ms/step - loss: 0.0047 - mae: 0.0514 - val_loss: 2.7761e-04 - val_mae: 0.0126
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 229s 355ms/step - loss: 0.0031 - mae: 0.0413 - val_loss: 0.0010 - val_mae: 0.0203
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 223s 347ms/step - loss: 0.0023 - mae: 0.0362 - val_loss: 5.3102e-04 - val_mae: 0.0146
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 234s 363ms/step - loss: 0.0020 - mae: 0.0334 - val_loss: 4.8421e-04 - val_mae: 0.0157
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 230s 357ms/step - loss: 0.0017 - mae: 0.0312 - val_loss: 0.0012 - val_mae: 0.0180
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 231s 358ms/step - loss: 0.0016 - mae: 0.0305 - val_loss: 9.7027e-04 - val_mae: 0.0169
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 228s 354ms/step - loss: 0.0015 - mae: 0.0293 - val_loss: 7.6012e-04 - val_mae: 0.

✅Model Succesfully Trained and Store as 'lstm_model.h5' 
